In [ ]:
import pandas as pd
from pathlib import Path

csv_path = Path("data/secretariats.csv")
if not csv_path.exists():
    raise FileNotFoundError(f"Cannot find {csv_path} in working directory: {Path.cwd()}")

gdf = pd.read_csv(csv_path, encoding="utf-8")
gdf.head()

,district_name,district_code,mandal_name,mandal_code,secretariat_code,secretariat_name,latitude,longitude
0,ALLURI SITHARAMA RAJU,745,ADDATEEGALA (R),4887,10490858,GONDOLU,17.450918,82.064411
1,ALLURI SITHARAMA RAJU,745,ADDATEEGALA (R),4887,10490857,GAVARAYYAPETA,17.461271,82.111261
2,ALLURI SITHARAMA RAJU,745,ADDATEEGALA (R),4887,10490860,KIMMURU,17.349885,82.127516
3,ALLURI SITHARAMA RAJU,745,ADDATEEGALA (R),4887,10490868,ADDATEEGALA2,17.480297,82.023781
4,ALLURI SITHARAMA RAJU,745,ADDATEEGALA (R),4887,10490856,DUCHARTHI,17.584431,82.057240


In [128]:
# add two columns 'unit' and 'gen_name' with former representing the place name and the latter
# Grama Sachivalayam or Ward Sachivalayam based on whether the mandal is rural or urban
gdf['unit']=gdf['secretariat_name']
# replace the 'unit' column  values matching with  r'().+)-0([0-9]{2,})' with r'\1-\2' 
gdf['unit'] = gdf['unit'].str.replace(r'(.+)-0([1-9])', r'\1-\2', regex=True)
# replace the 'unit' column  values matching with  'r'(.*(?<!-))(\d+)$')' with r'\1-\2' 
gdf['unit'] = gdf['unit'].str.replace(r'(.*(?<!-))(\d+)$', r'\1-\2', regex=True)
# convert 'unit' column to title case
gdf['unit'] = gdf['unit'].str.title()
# add a column with name 'gen_name' and initialize it with 'Grama sachivalayam' for all rows
gdf['gen_name'] = 'Grama Sachivalayam'
# Change 'gen_name' column to 'Ward Sachivalayam' where 'mandal_name' ends with '(U)'
gdf.loc[gdf['mandal_name'].str.endswith('(U)'), 'gen_name'] = 'Ward Sachivalayam'   

In [129]:
import re
# replace suffixes -i or -I with -1
gdf['unit'] = gdf['unit'].str.replace(r'(.+)-I$', r'\1-1', regex=True, flags=re.IGNORECASE)
# replace suffixes -ii or -II with -2
gdf['unit'] = gdf['unit'].str.replace(r'(.+)-II$', r'\1-2', regex=True, flags=re.IGNORECASE)
# ' - 0-' with '-' in 'unit' column
gdf['unit'] = gdf['unit'].str.replace(r'(.+) - 0-(.+)', r'\1-\2', regex=True, flags=re.IGNORECASE)
# ' - -' with '-' in 'unit' column
gdf['unit'] = gdf['unit'].str.replace(r'(.+) - -(.+)', r'\1-\2', regex=True, flags=re.IGNORECASE)

In [ ]:
# output gdf to secretariats_cleanedup.csv with utf-8 encoding
gdf.to_csv("data/secretariats_cleanedup.csv", index=False, encoding="utf-8")